<a href="https://colab.research.google.com/github/SoparAwayyy/Machine-Learning/blob/main/Week12/DWI_SAPUTRA_SOPAR_SIAGIAN_VGG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

DWI SAPUTRA SOPAR SIAGIAN

1103210220

Tugas : 12

Model: CNN Modern

Referensi: https://d2l.ai/chapter_convolutional-modern/vgg.html

In [ ]:
pip install d2l


In [ ]:
import torch
from torch import nn
import d2l.torch as d2l

In [ ]:
def vgg_block(num_convs, out_channels):
  # vgg_block. Fungsi ini menerima dua argumen: num_convs, yang merupakan jumlah lapisan konvolusi dalam blok
  # out_channels, yang menentukan jumlah saluran (channels) untuk keluaran dari setiap lapisan konvolusi.
    layers = []
    # membuat daftar kosong yang akan berisi lapisan-lapisan dari blok VGG.
    for _ in range(num_convs):
        layers.append(nn.LazyConv2d(out_channels, kernel_size=3, padding=1))
        # nn.LazyConv2d: Lapisan konvolusi dengan out_channels saluran, menggunakan kernel (filter) dengan ukuran 3x3 dan padding sebesar 1 piksel.
        layers.append(nn.ReLU())
        # Fungsi aktivasi ReLU yang akan diaplikasikan setelah setiap lapisan konvolusi.
    layers.append(nn.MaxPool2d(kernel_size=2,stride=2))
    #     # lapisan penurunan dimensi spasial (spatial downsampling) yang akan mengurangi dimensi spasial keluaran dari blok VGG dengan mengambil nilai maksimum dalam jendela kernel 2x2 dengan pergeseran (stride) 2.
    return nn.Sequential(*layers)
    #  untuk menggunakan blok VGG ini sebagai sebuah unit tunggal dalam membangun arsitektur jaringan yang lebih besar.


In [ ]:
class VGG(d2l.Classifier):
  # mengindikasikan bahwa model ini akan digunakan untuk klasifikasi.
    def __init__(self, arch, lr=0.1, num_classes=10):
      # metode konstruktor untuk kelas VGG. Ini menerima beberapa parameter:
        super().__init__()
        # memanggil konstruktor kelas dasar (d2l.Classifier).
        self.save_hyperparameters()
        # adalah metode dari PyTorch Lightning yang digunakan untuk menyimpan semua hiperparameter (termasuk yang dilewatkan ke __init__) ke dalam objek self.hparams.
        conv_blks = []
        # Membuat list kosong untuk menyimpan blok konvolusi dari model.
        for (num_convs, out_channels) in arch:
          # Melakukan iterasi pada setiap tuple dalam arch.
            conv_blks.append(vgg_block(num_convs, out_channels))
            # Menambahkan blok konvolusi baru ke dalam list conv_blks dengan menggunakan fungsi vgg_block yang belum didefinisikan.
        self.net = nn.Sequential(
            # Mendefinisikan model jaringan saraf sebagai objek nn.Sequential
            *conv_blks, nn.Flatten(),
            nn.LazyLinear(4096), nn.ReLU(), nn.Dropout(0.5),
            nn.LazyLinear(4096), nn.ReLU(), nn.Dropout(0.5),
            nn.LazyLinear(num_classes))
        self.net.apply(d2l.init_cnn)
        # Menerapkan inisialisasi CNN (convolutional neural network) dari modul d2l ke model, yang biasanya digunakan untuk menginisialisasi bobot dan bias dalam jaringan.


In [ ]:
VGG(arch=((1, 64), (1, 128), (2, 256), (2, 512), (2, 512))).layer_summary(
    (1, 1, 224, 224))
# untuk menganalisis arsitektur jaringan yang diberikan dengan ukuran input tertentu dan untuk menganalisis arsitektur dan ukuran input


In [ ]:
model = VGG(arch=((1, 16), (1, 32), (2, 64), (2, 128), (2, 128)), lr=0.01)
# Baris ini membuat model menggunakan arsitektur VGG
trainer = d2l.Trainer(max_epochs=10, num_gpus=1)
# menentukan jumlah epoch (iterasi penuh melalui seluruh dataset) yang akan dilakukan selama pelatihan.
data = d2l.FashionMNIST(batch_size=128, resize=(224, 224))
# Dataset ini akan digunakan untuk pelatihan model, menentukan ukuran batch yang akan digunakan selama pelatihan. , menentukan dimensi gambar yang akan diubah sebelum dimasukkan ke dalam model.
model.apply_init([next(iter(data.get_dataloader(True)))[0]], d2l.init_cnn)
# Baris ini menginisialisasi parameter model menggunakan metode apply_init
trainer.fit(model, data)
# proses pelatihan model menggunakan objek trainer yang telah dibuat sebelumnya.
# Metode fit dari objek trainer digunakan untuk melatih model menggunakan data yang telah diberikan.
# Proses pelatihan akan berlangsung selama jumlah epoch yang telah ditentukan.
